In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import helpers
import implementations

In [ ]:
PATH_TO_DATA = "input path here"

In [ ]:
x_train, x_test, y_train, train_ids, test_ids = helpers.load_csv_data(PATH_TO_DATA, sub_sample=False)

Here we should have some preprocessing/cleaning before running our implementations

In [ ]:

# Initialize parameters
initial_w = np.zeros(x_train.shape[1])  # Initialize weights to zero, shape (D,)
max_iters = 1000  # Number of iterations
gamma = 0.01  # Learning rate

# Run the gradient descent algorithm on the training data
w, final_loss = implementations.mean_squared_error_gd(y_train, x_train, initial_w, max_iters, gamma)

print("Final weights: ", w)
print("Final training loss: ", final_loss)

# Visualize loss over iterations
def plot_loss_over_time(losses):
    plt.plot(losses)
    plt.xlabel('Iteration')
    plt.ylabel('MSE Loss')
    plt.title('Training Loss over Iterations')
    plt.show()

# Predictions and visualization
def visualize_predictions(x, y_true, w):
    """Visualize the predicted vs true values."""
    y_pred = x.dot(w)
    plt.scatter(y_true, y_pred)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red')  # Diagonal line
    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title('True vs Predicted Values')
    plt.show()

# Visualize the results on the training set
visualize_predictions(x_train, y_train, w)

# Visualize the results on the test set
# visualize_predictions(x_test, y_test, w)